# Competición titanic con keras 


## importar librerias y datos

In [60]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

## Unir los dataframes.

In [61]:
data = pd.concat([train_data, test_data])

## Ingeniera de datos

In [62]:
y_train = data[:891].Survived

# Borrar columnas innesesarias.
features_to_remove = ['PassengerId', 'Ticket', 'Cabin',  'Survived']
data = data.drop(features_to_remove , axis=1)
#  Reemplazar datos categóricos por  de sexo0s y  por 1s.
data['Sex'] = LabelEncoder().fit_transform(data['Sex'])

### Generación de nuevos atributos

A partir de ``SibSp`` y ``Parch`` generamos y reemplazamos lo anterior con un nuevo atributo para verificar si el pasajero arrivó solo o con algún pariente: ``IsAlone``.

In [64]:
data['IsAlone'] = 0
data.loc[ (data['SibSp'] + data['Parch']) == 0, 'IsAlone'] = 1 
data.drop(['SibSp', 'Parch'], axis=1, inplace=True)

A partir de Los prefijos de los nombres del los pasajeros, se genera un nuevo atributo que representa el prefijo de los nombres: ``Title``.


In [ ]:
# Extraer prefijo del nombre de las personas.
data['Title'] = data.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

# Reemplazar los titulos raros con uno mas abarcativo: 'Rare'.
data['Title'] = data['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
data['Title'] = data['Title'].replace(['Mlle','Ms'], 'Miss')
data['Title'] = data['Title'].replace('Mme', 'Mrs')
data['Title'] = LabelEncoder().fit_transform(data['Title'])
data.drop(['Name'], axis=1, inplace=True)

Rellenamos ``Age`` faltantes según la relación con ``Sex`` y ``Pclass``.

In [ ]:
guess_ages = np.zeros((2,3))

for i in range(0, 2): # itera con => 0,1 para sexo
    for j in range(0, 3):  # itera con => 1,2,3 para pclass
        guess_df = data[(data['Sex'] == i) & (data['Pclass'] == j+1)]['Age'].dropna()

        # Convierte el numero decimal al .5 mas cercano
        guess_ages[i,j] = round( guess_df.mean() *2 ) / 2

        data.loc[ (data.Age.isnull()) & (data.Sex == i) & (data.Pclass == j+1),'Age'] = guess_ages[i,j]

Rellenamos ``Fare`` con su mediana.

In [ ]:
data['Fare'].fillna(data['Fare'].dropna().median(), inplace=True)

Convertimos columna de datos categóricos a variables dummys.

In [ ]:
data = pd.get_dummies(data, columns=['Pclass', 'Embarked'], prefix=['Pclass','Embarked'])

Split y visualización de ``data``.

In [ ]:
X_train = data[:891]
test = data[891:]

## Definición de modelo

In [ ]:
def build_model():
  model = keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=[len(X_train.keys())]),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid') 
  ])

  model.compile(loss='binary_crossentropy',
                optimizer=Adam(learning_rate=0.01),
                metrics=['accuracy'])

  return model

model = build_model()
model.summary()

## Entrenamiento del modelo


In [ ]:
history = model.fit(
    X_train,
    y_train,
    verbose=2, epochs=100, validation_split=0.2)

## Predicciones

In [ ]:
preds = model.predict(test)

preds[preds > 0.5] = 1
preds[preds <= 0.5] = 0

## Exportar predicciones

In [ ]:
prediction = pd.DataFrame({'PassengerId': test_data.PassengerId.values, 'Survived': preds.ravel().astype('int64')})
prediction.to_csv('data/results.csv', index=False)

In [ ]:
prediction.head(5)